In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

EventTypes = ['noType', 'sessionStart', 'sessionEnd', 'gameStart', 'gameEnd', 'roundStart', 'cardStateChange']
CardStates = ['left', 'right', 'center', 'dropped']

data = pd.read_json('../Data/ArchivoGuardado', lines=True)
display(data)

,EventType,TimeStamp,CardState,Win
0,1,2024-04-23 14:24:07.069,NaN,NaN
1,3,2024-04-23 14:24:39.255,NaN,NaN
2,6,2024-04-23 14:24:58.445,2.0,NaN
3,6,2024-04-23 14:24:58.533,1.0,NaN
4,5,2024-04-23 14:24:58.607,NaN,NaN
...,...,...,...,...
155,6,2024-04-23 14:28:01.701,3.0,NaN
156,6,2024-04-23 14:28:02.533,0.0,NaN
157,5,2024-04-23 14:28:08.034,NaN,NaN
158,4,2024-04-23 14:28:08.281,NaN,1.0


In [2]:
#METRICA: TIEMPO MEDIO EN CADA PREGUNTA

rounds = data[(data['EventType'] == 5) | (data['EventType'] == 4)]

rounds = rounds.reset_index(drop=True)

rounds['TimeDiff'] = rounds['TimeStamp'].diff()
indices = rounds[(rounds['EventType'] == 4)]['TimeDiff'].index

rounds = rounds.drop((indices + 1)[:-1])

media = rounds[(rounds['EventType'] == 5)]['TimeDiff'].mean()

print('MEDIA DE TIEMPO ENTRE RONDAS: \t', media)

MEDIA DE TIEMPO ENTRE RONDAS: 	 0 days 00:00:05.959464285


In [3]:
#METRICA: TIEMPO MEDIO QUE SE MANTIENE UNA CARTA EN CUALQUIERA DE LOS LADOS

cardsOnSide = data[(data['EventType'] == 6)]

cardsOnSide = cardsOnSide.reset_index(drop=True)

cardsOnSide['TimeDiff'] = cardsOnSide['TimeStamp'].diff()

indices = cardsOnSide[(cardsOnSide['CardState'] == 1) | (cardsOnSide['CardState'] == 0)].index
cardsOnSide = cardsOnSide.iloc[(indices + 1)[:-1]]

media = cardsOnSide['TimeDiff'].mean()

print('MEDIA DE TIEMPO QUE SE MANTIENE UNA CARTA A UN LADO: \t', media)

MEDIA DE TIEMPO QUE SE MANTIENE UNA CARTA A UN LADO: 	 0 days 00:00:02.817254237


In [5]:
#METRICAS: MEDIA DE VECES QUE SE ALTERNA A DERECHA E IZQUIERDA UNA CARTA 
#Y PORCENTAJE DE RONDAS EN LAS QUE SE ALTERNA DE DERECHA A IZQUIERDA

cardsAlternated = data[(data['EventType'] == 5)| (data['EventType'] == 6)]

n_rounds = 0
rounds_alternated = 0
n_alternated = 0
left = False
right = False
alternated_this_round = False

for i in range(len(cardsAlternated)):
    current_event_type = cardsAlternated.iloc[i]['EventType']
    
    #evento de inicio de ronda
    if current_event_type == 5:
        n_rounds += 1
        if alternated_this_round:
            rounds_alternated += 1
            
        right = False
        left = False
    else:
        current_card_state = cardsAlternated.iloc[i]['CardState']
        
        if current_card_state == 0:
            left = True
        elif current_card_state == 1:
            right = True

        if left and right:
            alternated_this_round = True
            n_alternated += 1
            
            if current_card_state == 0:
                right = False
            elif current_card_state == 1:
                left = False
            
print('MEDIA DE VECES QUE SE ALTERNA ENTRE LADOS POR RONDA: ', n_alternated/n_rounds)
print('PORCENTAJE DE RONDAS EN LAS QUE SE ALTERNA DE LADOS: ', rounds_alternated/n_rounds * 100, '%')

MEDIA DE VECES QUE SE ALTERNA ENTRE LADOS POR RONDA:  1.0
PORCENTAJE DE RONDAS EN LAS QUE SE ALTERNA DE LADOS:  93.33333333333333 %
